In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
import numpy as np

In [22]:
book_path = "/content/drive/MyDrive/DataMML/Books.csv"
ratings_path = "/content/drive/MyDrive/DataMML/Ratings.csv"
users_path = "/content/drive/MyDrive/DataMML/Users.csv"

In [23]:
# Books CSV
books = pd.read_csv(book_path , error_bad_lines=False )
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns = {'Publisher' : 'publisher'},inplace = True)
books.rename(columns = {'Book-Title' : 'title'},inplace = True)
books.rename(columns = {'Book-Author' : 'author'},inplace = True)
books.rename(columns = {'Year-Of-Publication' : 'year'},inplace = True)

<ipython-input-23-6eacfc7f5630>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(book_path , error_bad_lines=False )
<ipython-input-23-6eacfc7f5630>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(book_path , error_bad_lines=False )


In [24]:
# Users CSV
users =  pd.read_csv(users_path , error_bad_lines=False )
users.rename(columns = {'User-ID' : 'user-id' , 'Location' : 'location'  , 'Age' : 'age'},inplace = True)

<ipython-input-24-5802168362c2>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users =  pd.read_csv(users_path , error_bad_lines=False )


In [25]:
# Rating CSV
ratings =  pd.read_csv(ratings_path , error_bad_lines=False )
ratings.rename(columns = {'User-ID' : 'user-id' , 'Book-Rating' : 'rating'},inplace = True)

<ipython-input-25-ba0cb6a0e8db>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings =  pd.read_csv(ratings_path , error_bad_lines=False )


In [26]:
print(books.shape)
print(users.shape)
print(ratings.shape)

(271360, 5)
(278858, 3)
(1149780, 3)


Filtering ratings to include users who have rated more than 200 books and books with at least 50 ratings.

In [27]:
x = ratings['user-id'].value_counts()>200
y= x[x].index
ratings = ratings[ratings['user-id'].isin(y)]
ratings_with_books =  ratings.merge(books , on = 'ISBN')

In [28]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns ={'rating':'number of ratings'} , inplace =True)
final_rating = ratings_with_books.merge(number_rating , on = 'title')
new_final_rating = final_rating[final_rating['number of ratings']>=50]
new_final_rating.drop_duplicates(['user-id' , 'title'] , inplace = True)

<ipython-input-28-91ed4de2b67b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_final_rating.drop_duplicates(['user-id' , 'title'] , inplace = True)


In [29]:
book_pivot = new_final_rating.pivot_table(columns = 'user-id' , index = 'title' , values = 'rating')
book_pivot.fillna(0 , inplace = True)

In [30]:
book_pivot.shape

(742, 888)

Creating a sparse matrix from the pivot table using csr_matrix.

In [31]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)
type(book_sparse)

scipy.sparse._csr.csr_matrix

Using NearestNeighbors from scikit-learn to build the recommendation model.

In [32]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors( algorithm = 'brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

Defining a function recommend_books(book_name) that recommends similar books based on the input book name.

In [33]:
def recommend_books(book_name):
  book_id = np.where(book_pivot.index == book_name)[0][0]
  distances , suggestions = model.kneighbors (book_pivot.iloc[book_id , :].values.reshape(1, -1) , n_neighbors = 7)

  for i in range(len(suggestions)):
    if i ==0:
      print("The sugessions for" , book_name , "are :")
    if not i :
      print(book_pivot.index[suggestions[i]])

In [34]:
recommend_books('Jacob Have I Loved')

The sugessions for Jacob Have I Loved are :
Index(['Jacob Have I Loved', 'Exclusive', 'No Safe Place',
       'The Cradle Will Fall', 'Long After Midnight', 'Table For Two',
       'Alaska'],
      dtype='object', name='title')
